In [1]:
from sage.all import *
import pickle

sx, sy = var('sx, sy')

ox, oy = var('ox, oy')

dx = ox - sx
dy = oy - sy
r2 = (dx ** 2) + (dy ** 2)

r = sqrt(r2)
G = (1 / (2 * pi)) * log(r)

In [2]:
forget()
assume(sy-oy>0)
assume(sy > 0)
assume(sy < 1)
Ia = integral(G, sx, -1, 1)

In [3]:
assume(ox > -1)
assume(ox < 1)
assume(oy > -1)
coincident = integral(Ia, sy, -1, 1)

In [4]:
forget()
assume(ox > 1)
assume(oy > 1)
nearfield = integral(Ia, sy, -1, 1)

In [5]:
import pickle
with open('data/constant_test_integral.pkl', 'wb') as f:
    pickle.dump([coincident._sympy_(), nearfield._sympy_()], f)

In [85]:
forget()
assume(sy-oy>0)
assume(sy > 0)
assume(sy < 1)
Ia = integral(G * (1 - (1 - sx) ** 3) * (1 - (sy + 1) ** 2), sx, -1, 1)

In [86]:
assume(ox > -1)
assume(ox < 1)
assume(oy > -1)
coincident = integral(Ia, sy, -1, 1)

In [87]:
forget()
assume(ox > 1)
assume(oy > 1)
nearfield = integral(Ia, sy, -1, 1)

In [90]:
import pickle
with open('data/xy_test_integral.pkl', 'wb') as f:
    pickle.dump([coincident._sympy_(), nearfield._sympy_()], f)

In [84]:
for integrand, name in [(G, 'constant'), (G * (1 - sx) * (1 - sy ** 2), 'xy')]:
    
    transformed_nearfield = []
    for mult in [1, 2, 4, 8, 16]:
        forget()
        src_c = [0, 0]
        src_half_size = mult / 8
        assume(sy-oy > 0)
        Ia = integral(integrand, sx, src_c[0] - src_half_size, src_c[0] + src_half_size)
        assume(ox - (src_c[0] + src_half_size) > 0)
        assume(oy - (src_c[1] + src_half_size) > 0)

        transformed_nearfield.append(integral(Ia, sy, src_c[1] - src_half_size, src_c[1] + src_half_size))

    import pickle
    with open(f'data/{name}_transformed_test_integral.pkl', 'wb') as f:
        pickle.dump([I._sympy_() for I in transformed_nearfield], f)